<a href="https://colab.research.google.com/github/Pey-crypto/LOCI/blob/main/Initial_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install gdal-bin python-gdal python3-gdal
!sudo apt install install python3-rtree
!pip install git+https://github.com/geopandas/geopandas.git
!pip install descartes
!pip install folium
!pip install "dask[complete]"
!pip install plotly_express

Above listed are the requirements. Run that cell

In [ ]:
!ls -all
!rm -rf yelp_academic_dataset_business.json
!rm -rf yelp_academic_dataset_business.zip
!wget https://www.dropbox.com/s/3x1w789mmuae3ao/yelp_academic_dataset_business.zip
!unzip yelp_academic_dataset_business.zip
!ls -all


In [ ]:
import dask
import numpy as numpy
import geopandas as gpd
import matplotlib.pyplot as plpt
import seaborn as sns
import dask.dataframe as dd
import folium 
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly_express as px

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


In [ ]:
df = dd.read_json('yelp_academic_dataset_business.json')
df.head()

In [ ]:
df.shape

It is because when it is large number, Dask Delayed is activated

In [ ]:
df['Restaurants'] = df['categories'].str.contains('Restaurants')
df.head(5)

In [ ]:
df_restaurants = df.loc[df.Restaurants == True]
df_restaurants.head()

In [ ]:
df_restaurants.shape

In [ ]:
from sklearn.metrics import silhouette_score as sil
lasVegas = df_restaurants[df_restaurants.state == 'NV']
oods = lasVegas[['longitude','latitude']]
dist = []
silo = []
K = range(1,25)
kmax = 50
print("First calculation")
for k in K:
  km = KMeans(n_clusters = k)
  km = km.fit(oods)
  dist.append(km.inertia_)
  print("*",end="")
print("Second Calculation")
for k in range(2, kmax+1):
    km = KMeans(n_clusters = k).fit(oods)
    labels = km.labels_
    silo.append(sil(oods,labels,metric = 'euclidean'))
    print("*",end="")
    

#Attempt to thread these two functions
#Tasks remaining
#Identifying critical variables and values

"""

import _thread
import time

def k_means_fitting(delay):
  for k in K:
    time.sleep(delay)
    km = KMeans(n_clusters = k)
    km = km.fit(oods)
    dist.append(km.inertia_)
    count = count + 1
    print(count)

count=0
def k_means_auditing(delay):
  for k in range(2, kmax+1):
    time.sleep(delay)
    km = KMeans(n_clusters = k).fit(oods)
    labels = km.labels_
    silo.append(sil(oods,labels,metric = 'euclidean'))
    count = count + 1
    print(count)
try:
   print("deb")
   _thread.start_new_thread( k_means_fitting, (1, ) )
   _thread.start_new_thread( k_means_auditing, (2, ) )
except:
   print ("Error: unable to start thread")

"""




In [ ]:
km = KMeans(n_clusters=5,init='k-means++')
km.fit(oods)
y = km.labels_
print("sil score",sil(oods,y,metric='euclidean'))
lasVegas['cluster'] = km.predict(lasVegas[['longitude','latitude']])
lasVegas.head()